# Loading data

In [1]:
import warnings
warnings.filterwarnings('ignore')
from tensorboard.plugins.hparams import api as hp

# Dataset hyper parameters
HP_BALANCED = hp.HParam('balanced', hp.Discrete([0, 1])) # 0 unbalanced, 1 balanced
HP_NORM_IMAGES = hp.HParam('norm_images', hp.Discrete([0, 1]))

# Network hyper parameters
HP_NETWORKS = hp.HParam('network', hp.Discrete(['Xception',
                                            # 'VGG16',
                                            # 'VGG19',
                                            # 'ResNet50',
                                            'ResNet50V2',
                                            # 'ResNet101',
                                            # 'ResNet101V2',
                                            # 'ResNet152',
                                            'ResNet152V2',
                                            'InceptionV3',
                                            # 'InceptionResNetV2',
                                            # 'MobileNet',
                                            'MobileNetV2',
                                            'DenseNet121',
                                            # 'DenseNet169',
                                            # 'DenseNet201',
                                            # 'NASNetMobile',
                                            'NASNetLarge',
                                            'EfficientNetB0',
                                            # 'EfficientNetB1',
                                            # 'EfficientNetB2',
                                            # 'EfficientNetB3',
                                            # 'EfficientNetB4',
                                            # 'EfficientNetB5',
                                            # 'EfficientNetB6',
                                            'EfficientNetB7',
                                            'EfficientNetV2B0',
                                            'EfficientNetV2B1',
                                            # 'EfficientNetV2B2',
                                            # 'EfficientNetV2B3',
                                            # 'EfficientNetV2S',
                                            # 'EfficientNetV2M',
                                            'EfficientNetV2L',
                                            'ConvNeXtTiny',
                                            # 'ConvNeXtSmall',
                                            # 'ConvNeXtBase',
                                            # 'ConvNeXtLarge',
                                            # 'ConvNeXtXLarge' 
                                               ])) 


HP_ACTIVATION_FUNCTIONS = hp.HParam('act_functions', hp.Discrete(['relu', 'selu', 'tanh']))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64, 128]))
HP_EARLY_STOP = hp.HParam('early_stop', hp.Discrete([3, 5, 7, 10]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

2022-10-27 21:06:57.133422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 21:06:57.287809: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-27 21:06:57.323753: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 21:06:57.955373: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import tensorflow as tf
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)


def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        result_metrics = train_test_model(hparams)
        tf.summary.scalar(METRIC_NAMES, result_metrics, step=1)

In [3]:
from tensorflow.keras import datasets, layers, models

# Function for normalizing images
def normImages(X):
    for i, image in enumerate(X):
        max_n = np.max(image)
        image /= max_n
        X[i] = np.abs(image)
    return X

# Obtaining training, validation and test data
def train_val_test_split(balanced, normalized):
    spectrogram = ['spectrogram', 'mel', 'mtf']
    
    X_train = []
    X_test = []
    X_val = []

    y_flag = 0 # y's has not been obtained yet 
    # root_dir = "/home/polivares/scratch/Datasets/PowerLineFaults/"
    root_dir = '/home/polivares/Dropbox/Work/PostDoc/PowerLineFaults/'
    
    for sp in spectrogram:
        X_full = np.load(f"{root_dir}dataset_{sp}/full/images_full.npy")
        
        
        if not y_flag:
            y_flag = 1
            
            y_full = np.load(f"{root_dir}dataset_{sp}/full/labels_full.npy").reshape(-1)
            if balanced: # getting balanced data from index
                # Index 1, partial discharge
                index_1 = np.where(y_full==1)[0]
                len_index_1 = len(index_1)
                index_train_1, index_val_1, index_test_1 = index_1[:len_index_1//3], index_1[len_index_1//3:2*len_index_1//3], index_1[2*len_index_1//3:4*len_index_1//3]

                # Index 0, non partial discharge
                index_0 = np.where(y_full==0)[0]
                index_train_0, index_val_0, index_test_0 = index_0[:len_index_1//3], index_0[len_index_1//3:2*len_index_1//3], index_0[2*len_index_1//3:4*len_index_1//3]

                # Obtaining index
                index_train = np.concatenate([index_train_0, index_train_1])
                np.random.shuffle(index_train)
                index_val = np.concatenate([index_val_0, index_val_1])
                np.random.shuffle(index_val)
                index_test = np.concatenate([index_test_0, index_test_1])
                np.random.shuffle(index_test)

            else: # Unbalanced data, similar to the original from index
                index_full = np.arange(len(y_full))
                np.random.shuffle(index_full)
                len_index = 1000
                
                # Obtaining index
                index_train, index_val, index_test = index_full[:len_index], index_full[len_index:2*len_index], index_full[2*len_index:]
            
            y_train = y_full[index_train]
            y_val = y_full[index_val]
            y_test = y_full[index_test]
            
            del y_full
        
        if normalized:
            X_full = normImages(X_full)
        
        X_train.append(X_full[index_train])
        X_val.append(X_full[index_val])
        X_test.append(X_full[index_test])
        
        del X_full
        
    X_train_c = np.concatenate(X_train, axis=3) 
    X_val_c = np.concatenate(X_val, axis=3)
    X_test_c = np.concatenate(X_test, axis=3)
    
    return X_train_c, y_train, X_val_c, y_val, X_test_c, y_test


# X_train_c, y_train, X_val_c, y_val, X_test_c, y_test = train_val_test_split(balanced=0,normalized=0)

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

def train_test_model(hparams, METRICS, data):
    # Data
    X_train, y_train, X_val, y_val, X_test, y_test = data
    
    # Model creation
    print("Model creation")
    base_model = getattr(tf.keras.applications, hparams[HP_NETWORKS])(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    x = tf.keras.layers.Conv2D(64, (3,3), activation=hparams[HP_ACTIVATION_FUNCTIONS])(base_model.output)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=hparams[HP_ACTIVATION_FUNCTIONS])(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False # Esto impide que las capas se re entrenen
    
    # Compile model
    print("Model compile")
    early_stopping = EarlyStopping(patience = hparams[HP_EARLY_STOP])
    model.compile(loss = 'bce', optimizer = hparams[HP_OPTIMIZER], metrics=METRICS)
    
    # Obtaining data 
    # print("Obtaining data")
    # X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(hparams[HP_BALANCED], hparams[HP_NORM_IMAGES])
    
    # Fitting training
    print("Fitting training")
    history_model = model.fit(X_train, y_train, epochs=1000, 
                            validation_data=(X_val, y_val),
                            batch_size=10,
                            callbacks=[early_stopping])
    # Evaluation on test
    print("Evaluation on test")
    results = model.evaluate(X_test, y_test)
    
    del model
    # Returning metrics results
    print("Returning metrics results")
    return results

In [5]:
# Run evaluation with hparams
def run(run_dir, hparams, data):
    METRICS = [
          tf.keras.metrics.TruePositives(name='tp'),
          tf.keras.metrics.FalsePositives(name='fp'),
          tf.keras.metrics.TrueNegatives(name='tn'),
          tf.keras.metrics.FalseNegatives(name='fn'), 
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          tf.keras.metrics.AUC(name='auc'),
          tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
    
    METRICS_NAMES = [
        'loss',
        'tp',
        'fp',
        'tn',
        'fn',
        'accuracy',
        'precision',
        'recall',
        'auc',
        'prc'
    ]

    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        results = train_test_model(hparams, METRICS, data)
    
        for name, metric in zip(METRICS_NAMES, results):
            print(f"Summary: metric {name} value {metric}")
            tf.summary.scalar(name, metric, step=1)

In [6]:
session_num = 0

print("Obtaining data")
# data = train_val_test_split(hparams[HP_BALANCED], hparams[HP_NORM_IMAGES])
data = train_val_test_split(1, 1)
# data = train_val_test_split(1, 0)
# data = train_val_test_split(0, 1)
# data = train_val_test_split(0, 0)

# for balanced in HP_BALANCED.domain.values:
#     for norm_image in HP_NORM_IMAGES.domain.values:

for act_func in HP_ACTIVATION_FUNCTIONS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            for early_stop in HP_EARLY_STOP.domain.values:
                for optimizer in HP_OPTIMIZER.domain.values:
                    for network in HP_NETWORKS.domain.values:
                        tf.keras.backend.clear_session()
                        hparams = {
                             HP_BALANCED : 1,
                             HP_NORM_IMAGES: 1,
                             HP_NETWORKS: network,
                             HP_ACTIVATION_FUNCTIONS: act_func,
                             HP_NUM_UNITS: num_units,
                             HP_DROPOUT: dropout,
                             HP_EARLY_STOP: early_stop,
                             HP_OPTIMIZER: optimizer
                        }
                        run_name = "run-%d" % session_num
                        print('--- Starting trial: %s' % run_name)
                        print({h.name: hparams[h] for h in hparams})
                        run('logs/hparam_tuning/' + run_name, hparams, data)
                        session_num += 1

Obtaining data
--- Starting trial: run-0
{'balanced': 0, 'norm_images': 0, 'network': 'ConvNeXtTiny', 'act_functions': 'relu', 'num_units': 16, 'dropout': 0.2, 'early_stop': 3, 'optimizer': 'adam'}


2022-10-27 21:07:03.750924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 21:07:03.772598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 21:07:03.772696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-27 21:07:03.773603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Model creation
Model compile
Fitting training
Epoch 1/1000


2022-10-27 21:07:08.610166: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8300
2022-10-27 21:07:10.059557: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f529eade780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-10-27 21:07:10.059576: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2022-10-27 21:07:10.262443: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-10-27 21:07:10.262977: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


35/35 [==============================] - 10s 130ms/step - loss: 0.8375 - tp: 63.0000 - fp: 50.0000 - tn: 125.0000 - fn: 112.0000 - accuracy: 0.5371 - precision: 0.5575 - recall: 0.3600 - auc: 0.5188 - prc: 0.5010 - val_loss: 0.7123 - val_tp: 175.0000 - val_fp: 175.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000 - val_auc: 0.4500 - val_prc: 0.4522
Epoch 2/1000
35/35 [==============================] - 3s 99ms/step - loss: 0.7023 - tp: 17.0000 - fp: 17.0000 - tn: 158.0000 - fn: 158.0000 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.0971 - auc: 0.5090 - prc: 0.5115 - val_loss: 0.6931 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 175.0000 - val_fn: 175.0000 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.5000
Epoch 3/1000
35/35 [==============================] - 3s 99ms/step - loss: 0.7000 - tp: 10.0000 - fp: 8.0000 - tn: 167.0000 - fn: 165.0000 - accuracy

2022-10-27 22:03:28.490828: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 192.00MiB (rounded to 201326592)requested by op model/block_1_expand_BN/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-10-27 22:03:28.491234: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-10-27 22:03:28.491242: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 1462, Chunks in use: 1462. 365.5KiB allocated for chunks. 365.5KiB in use in bin. 122.2KiB client-requested in use in bin.
2022-10-27 22:03:28.491246: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 512, Chunks in use: 512. 316.8KiB allocated for chunks. 316.8KiB in use in bin. 266.6KiB client-requeste

ResourceExhaustedError: Graph execution error:

Detected at node 'model/block_1_expand_BN/FusedBatchNormV3' defined at (most recent call last):
    File "/home/polivares/anaconda3/envs/partial_discharge/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/polivares/anaconda3/envs/partial_discharge/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/polivares/anaconda3/envs/partial_discharge/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/polivares/anaconda3/envs/partial_discharge/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/polivares/anaconda3/envs/partial_discharge/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/polivares/anaconda3/envs/partial_discharge/